In [1]:
import os
from transformers import TrainingArguments
from Geneformer.geneformer import Classifier
from Geneformer.geneformer import TranscriptomeTokenizer


/c4/home/tagashe/jdmwork/jdmproject/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# tk = TranscriptomeTokenizer(
#   custom_attr_name_dict={
#     "disease_group": "disease_group",
#     # "sex": "sex",
#     # "self_reported_ethnicity": "self_reported_ethnicity",
#     "donor_id": "donor_id",
#     # "observation_joinid": "observation_joinid"
#   },
#   nproc=16
# )


# tk.tokenize_data(
#     "dataset",
#     "tokenized_dataset",
#     "tokenized",
#     file_format = "h5ad"  
# )

Tokenizing dataset/CD4_prepped.h5ad


100%|██████████| 17/17 [00:02<00:00,  6.64it/s]
/c4/home/tagashe/jdmwork/jdmproject/lib64/python3.11/site-packages/anndata/_core/merge.py:1434: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(
/c4/home/tagashe/jdmwork/jdmproject/lib64/python3.11/site-packages/anndata/_core/anndata.py:787: ImplicitModificationWarning: Trying to modify index of attribute `.obs` of view, initializing view as actual.
  getattr(self, attr).index = value
/c4/home/tagashe/jdmwork/jdmproject/lib64/python3.11/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/c4/home/tagashe/jdmwork/Geneformer/geneformer/tokenizer.py:511: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position

dataset/CD4_prepped.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


In [2]:
from transformers.trainer_utils import IntervalStrategy

import os

input_data_folder = os.path.abspath("tokenized_dataset/tokenized.dataset")
prepared_data_folder = os.path.abspath("CD4_finetune_prepared")
results_folder = os.path.abspath("CD4_finetune_results")

training_args = TrainingArguments(
      num_train_epochs=1,              
      learning_rate=5e-5,
      per_device_train_batch_size=12,
      lr_scheduler_type='polynomial',
      warmup_steps=50,
      weight_decay=0.01,
      skip_memory_metrics=True,
      report_to="none",                  # Avoid logging integrations
      save_strategy="no",                # Don't try to save model checkpoints (avoids
      seed=73,
  )

# training_args = {
#     "num_train_epochs": 0.9,
#     "learning_rate": 0.000804,
#     "lr_scheduler_type": "polynomial",
#     "warmup_steps": 1812,
#     "weight_decay":0.258828,
#     "per_device_train_batch_size": 1,
#     "seed": 73,
# }

model = Classifier(
      classifier="cell",
      cell_state_dict={"state_key": "disease_group", "states": "all"},
      max_ncells=100,                  
      training_args=training_args.to_dict(),
      freeze_layers=4,                 
      num_crossval_splits=1,           # no cross-validation
      forward_batch_size=64,
      nproc=8,
      model_version="V2")

os.makedirs(prepared_data_folder, exist_ok=True)
os.makedirs(results_folder, exist_ok=True)

0	HC16
1	HC17
2	HC18
3	HC19
4	HC20
5	JDM1
6	JDM10
7	JDM11
8	JDM12
9	JDM13
10	JDM14
11	JDM15
12	JDM2
13	JDM3
14	JDM4
15	JDM5
16	JDM6
17	JDM7
18	JDM8
19	JDM9

In [3]:
train_ids = [5, 9, 14, 17, 4, 10, 1, 12, 0, 15, 19, 16, 12, 14, 5, 9] # all cases ( 3 HCs, 6 TNJDM, 3 Active, 4 Inactive)
eval_ids = [6, 7, 3, 11] # 1 case each
test_ids = [2, 8, 13, 18] # 1 case each 

train_test_id_split_dict = {"attr_key": "donor_id",
                            "train": train_ids+eval_ids,
                            "test": test_ids}


model.prepare_data(
      input_data_file=input_data_folder,
      output_directory=prepared_data_folder,
      output_prefix="jdm",
      split_id_dict=train_test_id_split_dict
)


In [5]:
train_valid_id_split_dict = {"attr_key": "donor_id",
                            "train": train_ids,
                            "eval": eval_ids}

model_path = os.path.abspath("./Geneformer/Geneformer-V2-104M")

metrics = model.validate(
        model_directory=model_path,
        prepared_input_data_file=f"{prepared_data_folder}/jdm_labeled_train.dataset",
        id_class_dict_file=f"{prepared_data_folder}/jdm_id_class_dict.pkl",
        output_directory=results_folder,
        output_prefix="jdm_classifier",
        split_id_dict=train_valid_id_split_dict,
        n_hyperopt_trials=1    )

mkdir: cannot create directory ‘/c4/home/tagashe/jdmwork/CD4_finetune_results/250627_geneformer_cellClassifier_jdm_classifier/’: File exists
  0%|          | 0/1 [00:00<?, ?it/s]

****** Validation split: 1/1 ******



2025-06-27 15:33:46,498	INFO worker.py:1917 -- Started a local Ray instance.
mkdir: cannot create directory ‘/c4/home/tagashe/jdmwork/CD4_finetune_results/250627_geneformer_cellClassifier_jdm_classifier/ksplit1’: File exists
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /c4/home/tagashe/jdmwork/Geneformer/Geneformer-V2-104M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/c4/home/tagashe/jdmwork/jdmproject/lib64/python3.11/site-packages/transformers/training_args.py:2080: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upg

== Status ==
Current time: 2025-06-27 15:33:51 (running for 00:00:00.11)
Using FIFO scheduling algorithm.
Logical resource usage: 0/104 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /scratch/tagashe/ray/session_2025-06-27_15-33-39_782521_1168859/artifacts/2025-06-27_15-33-51/_objective_2025-06-27_15-33-51/driver_artifacts
Number of trials: 1/1 (1 PENDING)




(_objective pid=1347034) /c4/home/tagashe/jdmwork/jdmproject/lib64/python3.11/site-packages/ray/train/_internal/session.py:834: RayDeprecationWarning: `ray.train.get_checkpoint` should be switched to `ray.tune.get_checkpoint` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(_objective pid=1347034)   _log_deprecation_warning(
(_objective pid=1347034) Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
(_objective pid=1347034) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /c4/home/tagashe/jdmwork/Geneformer/Geneformer-V2-104M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
(_objective pid=1347034) You should probab

Trial name
_objective_acb63b42


2025-06-27 15:34:40,470	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/c4/home/tagashe/jdmwork/CD4_finetune_results/250627_geneformer_cellClassifier_jdm_classifier/ksplit1/_objective_2025-06-27_15-33-51' in 0.0332s.
  0%|          | 0/1 [01:02<?, ?it/s]

== Status ==
Current time: 2025-06-27 15:34:40 (running for 00:00:49.15)
Using FIFO scheduling algorithm.
Logical resource usage: 8.0/104 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /scratch/tagashe/ray/session_2025-06-27_15-33-39_782521_1168859/artifacts/2025-06-27_15-33-51/_objective_2025-06-27_15-33-51/driver_artifacts
Number of trials: 1/1 (1 ERROR)
+---------------------+----------+-------------------------+-----------------+---------------------+--------------------+------------------------+---------+----------------+----------------+
| Trial name          | status   | loc                     |   learning_rate | lr_scheduler_type   |   num_train_epochs |   per_device_train_bat |    seed |   warmup_steps |   weight_decay |
|                     |          |                         |                 |                     |                    |                ch_size |         |                |                |
|---------------------+----------+------------------

TuneError: ('Trials did not complete', [_objective_acb63b42])

In [ ]:
metrics

{'conf_matrix':      0    1    2    3
 0  2.0  0.0  1.0  0.0
 1  6.0  0.0  1.0  0.0
 2  2.0  0.0  1.0  0.0
 3  0.0  0.0  0.0  0.0,
 'macro_f1': [0.21367521367521367],
 'acc': [0.23076923076923078],
 'all_roc_metrics': None}